Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
from pathlib import Path
# Add the directory containing your script to the Python path
sys.path.append(r'C:\Users\User\Project-SLMonlineControl\CodeAndPackages\PYsubfun')
#sys.path.append(r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS

In [2]:
#importlib.reload(FBS)
# Pre-set parameters, and load the only bin files in the pre-set path folde
ConfigFolder=r'C:\Users\User\Project-SLMonlineControl\config'
#ConfigFolder=r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\config'
ops0, confSet=FBS.configLoad(ConfigFolder,'SLMsetting.yml','ops3Planes.npy')
SaveFolder=FBS.suite2pInitiate(ops0,ConfigFolder)
binFile = glob.glob(ops0['save_path0'] + '/TSeries*-001.bin')    ##<<<<<<<<<<<<<<<<<<<<<<<<<< Noted that there supposed be only one Bin Data file
#Load Data
print(binFile)
rawBin, FramePerPlane, TotalFrameNeed=FBS.LoadBin(binFile,ops0)
rawBin.shape

Processed data would be saved inE:\LuSLMOnlineTest\SL0901\06052025\suite2p
File found: C:\Users\User\Project-SLMonlineControl\config\SLMsetting.yml
Copying to: E:\LuSLMOnlineTest\SL0901\06052025\CurrentSLMsetting.yml
Current SLMsetting copied successfully!
['E:\\LuSLMOnlineTest\\SL0901\\06052025\\TSeries-06052025-0907-001.bin']


(24600, 512, 512)

In [3]:
ops0['spatial_scale']=2
ops0['threshold_scaling']=1.6
ops0['denoise']=1

ops0

{'suite2p_version': '0.14.2.dev17+gf3d7d4e',
 'look_one_level_down': 1.0,
 'fast_disk': [],
 'delete_bin': False,
 'mesoscan': False,
 'bruker': False,
 'bruker_bidirectional': False,
 'h5py': [],
 'h5py_key': 'data',
 'nwb_file': '',
 'nwb_driver': '',
 'nwb_series': '',
 'save_path0': 'E:\\LuSLMOnlineTest\\SL0901\\06052025\\',
 'save_folder': [],
 'subfolders': [],
 'move_bin': False,
 'nplanes': 3,
 'nchannels': 1,
 'functional_chan': 1,
 'tau': 1.5,
 'fs': 6.9,
 'force_sktiff': False,
 'frames_include': -1,
 'multiplane_parallel': 0.0,
 'ignore_flyback': [],
 'preclassify': 0.0,
 'save_mat': True,
 'save_NWB': 0.0,
 'combined': 1.0,
 'aspect': 1.0,
 'do_bidiphase': False,
 'bidiphase': 0.0,
 'bidi_corrected': False,
 'do_registration': 1,
 'two_step_registration': 0.0,
 'keep_movie_raw': False,
 'nimg_init': 300,
 'batch_size': 500,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': True,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'smooth_sigma_time': 0.0,
 'smooth_sigma': 1.15,

Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [4]:
# Define the number of parallel processes; 
# Noted that current binning movie in chuns of length 06; increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes=1              ## <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Create a list of plane indices
plane_indices = range(0,ops0['nplanes'])
print(plane_indices)
ops1=ops0
Parallel(n_jobs=num_processes)(delayed(FBS.process_plane)(plane_idx, rawBin, SaveFolder, ops1) for plane_idx in plane_indices)

# Combine multiple plane data
#FBS.CombinePlanes(SaveFolder,ops0)
FBS.CombinePlanes(SaveFolder,ops0)

range(0, 3)
Processing plane 0
Binning movie in chunks of length 10
Binned movie of size [820,512,512] created in 7.43 sec.
Binned movie denoised (for cell detection only) in 16.50 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 16.00 
0 ROIs, score=179.25
Detected 128 ROIs, 14.92 sec
After removing overlaps, 128 ROIs remain
added enhanced mean image
Masks created, 0.76 sec.


c:\users\user\suite2p\suite2p\extraction\extract.py:125: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  Fi[n] = np.dot(data[:, cell_ipix[n]], cell_lam[n])


Extracted fluorescence from 128 ROIs in 8200 frames, 15.70 sec.
Extracted fluorescence from 128 ROIs in 8200 frames, 10.38 sec.
['compact', 'npix_norm', 'skew']
Processing plane 1
Binning movie in chunks of length 10
Binned movie of size [820,512,512] created in 7.74 sec.
Binned movie denoised (for cell detection only) in 17.33 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 16.00 
0 ROIs, score=254.62
Detected 153 ROIs, 15.08 sec
After removing overlaps, 152 ROIs remain
added enhanced mean image
Masks created, 0.70 sec.
Extracted fluorescence from 152 ROIs in 8200 frames, 10.13 sec.
Extracted fluorescence from 152 ROIs in 8200 frames, 10.22 sec.
['compact', 'npix_norm', 'skew']
Processing plane 2
Binning movie in chunks of length 10
Binned movie of size [820,512,512] created in 7.69 sec.
Binned movie denoised (for cell detection only) in 15.68 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.00, threshold 16.00 
0 ROIs, score=193.62
Detected 112 ROIs, 

Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [5]:
FBS.PostMannual(SaveFolder,ops0)

383 of units including 267 identified cells in combined planes
